In [ ]:
!pip install xgboost==1.4.1

In [ ]:
!pip install pickle5

In [ ]:
!pip install scikit-learn==0.23.2

In [ ]:
# AWS Sagemaker
import sagemaker
import boto3
from sagemaker.session import s3_input, Session

In [ ]:
# AWS Sagemaker: Define bucket's name for project
bucket = 'ls-aishub-inflated'
my_region = boto3.session.Session().region_name
print(f"Region: {my_region}")

# Get permission to read from S3 buckets
from sagemaker import get_execution_role
role = get_execution_role()
print(f"Role: {role}")

In [ ]:
import sklearn
import numpy as np
import pandas as pd
import xgboost as xgb
import pickle5 as pickle

print(f"Packages' versions:")
print(f"\tScikit-learn: {sklearn.__version__}")
print(f"\tNumpy: {np.__version__}")
print(f"\tPandas: {pd.__version__}")
print(f"\tXGBoost: {xgb.__version__}")

In [ ]:
def load_dev_csv(target, data='containershipdb_vu_devset.csv', **kwargs):
    print(f"Load data...")
    location = f's3://{bucket}/experimental_data/{data}'
    usecols_static = [
        "ais_dim_a",
        "ais_dim_b",
        "ais_dim_c",
        "ais_dim_d",
        "draught_fact",
    ] + [target]
    df = pd.read_csv(
        location,
        header=0,
        usecols=usecols_static,
        **kwargs,
    )
    return df

In [ ]:
def make_example_df(target):
    """Create example df for checking model prediction
    Model fitted on pandas data frame, due that convert to df
    """
    # Get instances from DEV set
    rows = [0, 600, 800]
    df = load_dev_csv(target=target, nrows=1000).iloc[rows, :]

    # With 1 data example
    example_df_1 = df.iloc[[0], :-1]
    y_true_1 = df.iloc[[0], -1].to_list()
    # With 2 data examples
    example_df_2 = df.iloc[[1, 2], :-1]
    y_true_2 = df.iloc[[1, 2], -1].to_list()

    examples = [(example_df_1, y_true_1), (example_df_2, y_true_2)]

    return examples

In [ ]:
def load_model(bucket, model_name):
    print(f"Load model...")
    model_path = f"models/vessel-utilization/{model_name}/model.pkl"
    model = "model.pkl"
    
    s3client = boto3.client('s3')
    s3client.download_file(bucket, model_path, model)

    with open(model, "rb") as f:
        model = pickle.load(f)
    return model

In [ ]:
def mape(y_true, y_pred):
    """Mean absolute percentage error regression loss.

    Examples
    --------
    >>> from sklearn.metrics import mean_absolute_percentage_error
    >>> y_true = np.array([3, -0.5, 2, 7])
    >>> y_pred = np.array([2.5, 0.0, 2, 8])
    0.3273809523809524...

    >>> y_true = np.array([1.0, -1.0, 2, -2])
    >>> y_pred = np.array([0.7, -0.7, 1.4, -1.4])
    >>> mean_absolute_percentage_error(y_true, y_pred)
    0.30000000000000004...
    """
    # Epsilon: is an arbitrary small yet strictly positive numbe
    # to avoid undefined results when y is zero
    epsilon = np.finfo(np.float64).eps
    ape = np.abs(y_pred - y_true) / np.maximum(np.abs(y_true), epsilon)
    return np.average(ape)

In [ ]:
def predict_example_df(model, target):
    examples = make_example_df(target=target)
    for e in examples:
        print(f"\nTest example w/ {len(e[0])} instances:")
        print(e[0])
        # Make prediction
        prediction = model.predict(e[0])
        mare = mape(e[1], prediction)
        # Print out
        print(f"\nReal Value: {e[1]}")
        print(f"Predicted Value: {list(prediction)}")
        print(f"Mean Absolute Ratio Error: {mare}\n")

In [ ]:
def main(target=None):
    """Checks a one-off prediction using the trained model.

    Args:
        * model_name (str): Folder's  name for separatly saved model.\
            Defaults to None.
    """
    print(f"--------------- START: Check an one-off prediction ----------------")
    if target is None:
        print(f"No target's name...")
        pass
    
    elif  target == "vu_estimated":
        print(f"Target is: <{target}>")
        model_name = "XGBRegressor-vu_estimated_0c1914e51e754b3893110f91279301f5"
    
    elif target == "teu_estimated":
        print(f"Target is: <{target}>")
        model_name = "XGBRegressor-teu_estimated_9ffee2d3fa3d469ea7bfc9df7e5d317f"
        
        
    model = load_model(bucket=bucket, model_name=model_name)
    predict_example_df(model, target)
    print(f"!!! DONE: an one-off prediction passed !!!")

In [ ]:
# To test perdiction for <vu_estimated>
main(target="vu_estimated")

In [ ]:
# To test perdiction for <teu_estimated>
main(target="teu_estimated")